# ロジスティック回帰他

### 今回利用するデータ

今回の演習では、Titanicの生存者情報データを用いる.
データは`titanic`パッケージに入っている`titanic_train`を用いる.

まずは, 変数を確認する

In [ ]:
library(pacman)
p_load(titanic)
names(titanic_train)

# data transformation

## 欠測値の処理

### 欠測値の定義

今回使うデータは空欄と`NA`が混在しているため, NAに統一する.
利用するのは`naniar::replace_with_na_all()`関数.

In [ ]:
p_load(naniar)
train <- replace_with_na_all(titanic_train,
                             condition = ~.x %in% c("NA", ""))

### データ構造の確認

データ構造を確認するため, `dplyr::glimpse()`関数を利用する.

In [ ]:
p_load(dplyr)
glimpse(train)

## 先頭から数行を確認

先頭から数行を確認するために`head()`関数を利用する.
`Cabin`の1行目と3行目は欠測値になっていることがわかる.

In [ ]:
head(train)

### データの概要を確認

データの記述統計を確認するために`summary()`関数を利用する.
文字列の欠測値は表示されない.

In [ ]:
summary(train)

### 欠測値を確認

このデータはいくつかの欠測値を含んでいるため,
欠測値がどの変数にいくつ含まれているのか確認する.
`dplyr::summarise()`関数を利用し,
`is.na`でNAに該当するセルがいくつあるのかを数え上げる.

In [ ]:
summarise(train,
          across(everything(),
                 \(x) sum(is.na(x))))

### 欠測値の除去

欠測値を含む行を削除する. これをリストワイズ除去という. ここでは,
`na.omit()`関数と`tidyr::drop_na()`関数の2種類を紹介する.

In [ ]:
train2 <- na.omit(train)

p_load(tidyr)
train2 <- tidyr::drop_na(train)

### 欠測値を除去したデータを確認

In [ ]:
head(train2)

In [ ]:
summary(train2)

## パイプ演算子

### コードを書いていて困ること1

`()`がいくつも重なるととてもコードが読みにくい.
`()`の一番内側から処理を実行し,
コードの先頭にある関数は一番最後に実行する関数となる. 例えば,
下の例にあるコードを実行する場合, `fun3(x)`を最初に実行し,
その結果に対して`fun2()`を実行し,
さらにその結果を`fun1()`が受け取って実行する.
これは人間の思考と逆である.

In [ ]:
fun1(fun2(fun3(x)))

### コードを書いていて困ること2

関数などをネストして書くことを避けると一時的に作成した中間的なオブジェクトが多くなる.
全くだめなわけではないが, 多すぎると見通しが悪い.

In [ ]:
x1 <- fun3(x)
x2 <- fun2(x1)
fun1(x2)

$\rightarrow$ これらの解決する手段として`|>`(パイプ)演算子がある

### `|>`(パイプ)とは

`|>`(パイプ)演算子を使うことで、パイプ演算子の前の結果をパイプ演算子の後の関数の第一引数に入れることができる.
パイプ演算子はR4.1.0から素のRに導入された.

以前のパイプ演算子`%>%`は`magrittr`パッケージの中の関数の1つ.
`dplyr`パッケージを読み込むと一緒に読み込んでくれる.

`|>`はctrl(cmd) + Shift(opt) + Mで書くことができる.
特に`dplyr`パッケージ等を用いたデータの前処理で大活躍する.

In [ ]:
fun3(x) |> 
  fun2() |> 
  fun1()

## データの整形

### 変数の変換

いくつかの変数のデータ型を`Integer`や`Character`から`factor`に,
`Character`から文字の抜き出しなどを行う.
列の変換には`dplyr::mutate()`関数を利用する

In [ ]:
p_load(stringr)
train3 <- train2 |> 
  mutate(across(c(Survived, Pclass, Sex, Embarked),
                as.factor)) |> 
  mutate(CabinLetter = str_sub(Cabin,
                               start = 1,
                               end   = 1) |> 
           as.factor()) |> 
  mutate(Title = str_sub(Name,
                         start = str_locate(Name, ",")[,1] + 2,
                         end   = str_locate(Name, "\\.")[,1] - 1) |> 
           as.factor())

# Logistic Regression Model

### ロジスティック回帰モデル

ロジスティック回帰モデルを推定するために`glm(family = binomial(link="logit"))`を利用する.
モデルを2つ作成し, 推定を行う.

In [ ]:
model1 <- Survived ~ Pclass + Sex + Age + 
  SibSp + Parch + Fare + Embarked +
  CabinLetter + Title
model2 <- Survived ~ Pclass + Sex + Age

### モデル1の結果

`jtools::summ()`関数を利用して結果の確認を行う.

In [ ]:
glm_logit1 <- glm(model1, 
                  data   = train3, 
                  family = binomial(link = 'logit'))
p_load(jtools)
summ(glm_logit1)

### 多重共線性のチェック

`summ()`関数の引数`vifs`を利用してモデル1の多重共線性をチェックする.
`Sex`と`Title`の値が大きく, 推定に問題があることが示唆される. `Sex`とMr,
Mrsなどを含む`Title`が相関するのは当然.

In [ ]:
summ(glm_logit1, vifs = TRUE)

### モデル2の結果

モデルを変えて(`CabinLetter`と`Title`を除いて)分析を行う.

In [ ]:
glm_logit2 <- glm(model2,
                  data   = train3, 
                  family = binomial(link = 'logit'))
summ(glm_logit2,
     vifs = TRUE)

### 推定された係数の図示

推定された係数を図で確認するために,
`coefplot::coefplot()`関数を利用する.

In [ ]:
p_load(coefplot)
coefplot(glm_logit2, 
         intercept = FALSE)

### オッズ比の算出

ロジスティック回帰モデルにより算出された回帰係数はそのままでは解釈できないため,
指数変換しオッズ比を計算する.

In [ ]:
summ(glm_logit2, exp = TRUE)

# オッズ比の理解

指数変換して計算したオッズ比がどのようなものかを理解する.

In [ ]:
p <- predict(glm_logit2,
        newdata = tibble(Pclass = factor(1),
                         Sex    = "male",
                         Age    = 35),
        type    = "response")
q <- predict(glm_logit2,
        newdata = tibble(Pclass = factor(2),
                         Sex    = "male",
                         Age    = 35),
        type    = "response")
(q / (1 - q)) / (p / (1 - p))

### オッズ比の図示

`ggplot2`を使ってオッズ比の信頼区間を図示する.

In [ ]:
p_load(broom)
broom::tidy(glm_logit2, 
            conf.int     = TRUE, 
            exponentiate = TRUE) |> 
  select(term, 
         estimate,
         conf.low, 
         conf.high) |> 
  filter(term != "(Intercept)") |> 
  ggplot() +
  aes(x    = term,
      y    = estimate,
      ymin = conf.low,
      ymax = conf.high) +
  geom_pointrange(size   = .5,
                  colour = "blue") +
  geom_hline(yintercept = 1,
             linetype   = 'dotted') +
  coord_flip()

### ロジスティック回帰モデルの残差チェック

残差をチェックするために`autoplot`を行う.

In [ ]:
p_load(ggfortify)
autoplot(glm_logit2)

### 評価用の関数作成

In [ ]:
perform <- function(data, observed, predict){
  library(dplyr)
  obs  <- enquo(observed)
  pred <- enquo(predict)
  result <- data |> 
    select(obs  := !!obs, 
           pred := !!pred) |> 
    mutate(TP = if_else(obs == 1 & pred == 1,
                        1, 0),
           FP = if_else(obs == 0 & pred == 1,
                        1, 0),
           TN = if_else(obs == 0 & pred == 0,
                        1, 0),
           FN = if_else(obs == 1 & pred == 0,
                        1, 0)) |> 
    summarise(across(c(TP, FP, TN, FN), sum)) |> 
    mutate(Pe = ((TP + FN) * (TP + FP) + (TN + FN) * (TN + FP)) /
             sum(TP, FP, TN, FN)^2) |> 
    transmute(Accuracy  = (TP + TN) / (TP + TN + FP + FN),
              Precision = TP / (TP + FP),
              Recall    = TP / (TP + FN),
              F1        = 2 * Precision * Recall / (Precision + Recall),
              kappa     = (Accuracy - Pe) / (1 - Pe))
  return(result)
}

### ロジスティックモデルを用いた予測と的中率

`predict`を用いてモデル上の(理論的な)生存確率を予測する.
予測された結果が0.5より大きい場合は生存, 0.5以下であれば死亡とする.
利用したデータを用いた予測の結果, 76.5%が的中した.

In [ ]:
train3 |>
  mutate(predict = predict(glm_logit2, 
                           type = "response")) |> 
  mutate(survive = if_else(predict > 0.5, 1, 0)) |> 
  perform(Survived, survive)

### LOOCVによる評価

`caret`パッケージを利用してLOOCVによるモデルの評価を行う. 正解率は75.4%.

In [ ]:
p_load(caret)
train(data      = train3,
      model2,
      method    = "glm",
      family    = binomial(link = "logit"),
      metric    = "Accuracy",
      trControl = trainControl(method = "LOOCV"))

# Probit Regression Model

### プロビット回帰モデル

プロビット回帰モデルを推定するためには,
`glm(family=binomial(link='probit'))`を指定する.

In [ ]:
glm_probit <- glm(model2, 
                  data   = train3,
                  family = binomial(link = 'probit'))
summ(glm_probit)

### プロビットモデルで推定された係数の図示

推定された係数を図で確認するために, `coefplot::coefplot`を利用する.

In [ ]:
coefplot(glm_probit, intercept = FALSE)

### プロビットモデルの残差チェック

残差をチェックするために`autoplot`を行う.

In [ ]:
autoplot(glm_probit)

### プロビットモデルを用いた予測と的中率

`predict`を用いてモデル上の(理論的な)生存確率を予測する.
予測された結果が0.5より大きい場合は生存, 0.5以下であれば死亡とする.
利用したデータを用いた予測の結果, 77.0%が的中した.

In [ ]:
train3 |>
  mutate(predict = predict(glm_probit, 
                           type = "response")) |> 
  mutate(survive = if_else(predict > 0.5, 1, 0)) |> 
  perform(Survived, survive)

### LOOCVによる評価

`caret`パッケージを利用してLOOCVによるモデルの評価を行う. 正解率は75.4%.

In [ ]:
train(data      = train3,
      model2,
      method    = "glm",
      family    = binomial(link = "probit"),
      metric    = "Accuracy",
      trControl = trainControl(method = "LOOCV"))

# Complementary Log-Log regression model

### 補対数対数回帰モデル

補対数対数回帰モデルを推定するためには,
`glm(family=binomial(link='cloglog'))`を指定する.

In [ ]:
glm_cloglog <- glm(model2, 
                   data   = train3,
                   family = binomial(link = 'cloglog'))
summ(glm_cloglog)

### 補対数対数モデルで推定された係数の図示

推定された係数を図で確認するために, `coefplot::coefplot`を利用する.

In [ ]:
coefplot(glm_cloglog, intercept = FALSE)

### 補対数対数モデルの残差チェック

残差をチェックするために`autoplot`を行う.

In [ ]:
autoplot(glm_cloglog)

### 補対数対数モデルを用いた予測と的中率

`predict`を用いてモデル上の(理論的な)生存確率を予測する.
予測された結果が0.5より大きい場合は生存, 0.5以下であれば死亡とする.
利用したデータを用いた予測の結果, 77.6%が的中した.

In [ ]:
train3 |>
  mutate(predict = predict(glm_cloglog, 
                           type = "response")) |> 
  mutate(survive = if_else(predict > 0.5, 1, 0)) |> 
  perform(Survived, survive)

### LOOCVによる評価

`caret`パッケージを利用してLOOCVによるモデルの評価を行う. 正解率は74.3%.

In [ ]:
train(data      = train3,
      model2,
      method    = "glm",
      family    = binomial(link = "cloglog"),
      metric    = "Accuracy",
      trControl = trainControl(method = "LOOCV"))

# Appendix

### テストデータによる予測

トレーニングデータで作成したモデルから, テストデータで予測する.

In [ ]:
test <- titanic_test |> 
  replace_with_na_all(condition = ~.x %in% c("NA", "")) |> 
  tidyr::drop_na() |> 
  mutate_at(vars(Pclass, Sex, Embarked), as.factor) |> 
  broom::augment(x            = glm_logit2,
                 newdata      = _,
                 type.predict = "response") |> 
  mutate(survive = if_else(.fitted > 0.5, 1, 0))

### 多重代入法による欠測値の処理

欠測値をバイアスをより少なく扱うためには, 多重代入法などを利用する.
`mice`, `norm2`, `miceadds`パッケージを利用する.

In [ ]:
p_load(mice, norm2, miceadds)
glm_logit_imp <- train |> 
  replace_with_na_all(condition = ~.x %in% c("NA", "")) |> 
  mutate(across(c(Pclass, Sex, Embarked), 
                as.factor)) |> 
  select(Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked) |> 
  mice(m    = 100,
       seed = 20191015,
       meth = c("", "", "", "norm", "", "", "norm", "polyreg")) |> 
  glm.mids(model2, 
           data   = _, 
           family = binomial(link = 'logit'))

### 多重代入法による結果の確認

多重代入法の結果をpoolした上で確認する.

In [ ]:
glm_logit_imp |> 
  pool() |> 
  summary()

### 多重代入法(ロジスティック回帰モデル)を用いた予測と的中率

利用したデータを用いた予測の結果, 79.3%が的中した.

In [ ]:
pooled_lm <-  glm_logit_imp$analyses[[1]]
pooled_lm$coefficients <-  summary(pool(glm_logit_imp))$estimate
train |> 
  mutate(predict = predict(pooled_lm, type = "response")) |> 
  mutate(survive = if_else(predict > 0.5, 1, 0)) |> 
  perform(Survived, survive)

### テストデータを用いた多重代入法による予測

正答率はkaggleにsubimitして確かめて見てください.

In [ ]:
test2 <- titanic_test |> 
  replace_with_na_all(condition = ~.x %in% c("NA", "")) |> 
  mutate(across(c(Pclass, Sex, Embarked),
                as.factor)) |> 
  select(Pclass, Sex, Age, SibSp, Parch, Fare, Embarked) |> 
  mice(m    = 100,
       seed = 31415926,
       meth = c("", "", "norm", "", "", "norm", "")) |> 
  complete() %>%
  mutate(predict = predict(pooled_lm,
                           type    = "response", 
                           newdata = .)) |> 
  mutate(survive = if_else(predict > 0.5, 1, 0))